# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124340e+02     1.958656e+00
 * time: 0.30257296562194824
     1     1.091122e+01     9.835741e-01
 * time: 1.8967039585113525
     2    -1.067184e+01     1.125602e+00
 * time: 2.4523110389709473
     3    -3.371219e+01     8.961819e-01
 * time: 3.274899959564209
     4    -4.701935e+01     6.697879e-01
 * time: 4.114284992218018
     5    -5.670420e+01     2.436583e-01
 * time: 4.92576003074646
     6    -5.975162e+01     2.183344e-01
 * time: 5.473284006118774
     7    -6.097215e+01     8.147592e-02
 * time: 6.0187249183654785
     8    -6.144886e+01     3.973755e-02
 * time: 6.575844049453735
     9    -6.169635e+01     5.507894e-02
 * time: 7.122575998306274
    10    -6.187304e+01     3.760769e-02
 * time: 7.671466827392578
    11    -6.200255e+01     2.629436e-02
 * time: 8.218660831451416
    12    -6.207963e+01     1.679311e-02
 * time: 8.765447854995728
    13    -6.211455e+01     1.728892e-02
 * time: 9.318011999130249
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557676
    AtomicNonlocal      14.8522644
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666462769
